<a href="https://colab.research.google.com/github/heatherstrelevitz/ht_workshop/blob/main/suite2p_visualizations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install numpy>=1.24
!pip install matplotlib

import numpy as np
from scipy.stats import zscore
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from matplotlib.patches import Polygon

In [ ]:
#load data
F = np.load('F.npy', allow_pickle=True)
Fneu = np.load('Fneu.npy', allow_pickle=True)
spks = np.load('spks.npy', allow_pickle=True)
stat = np.load('stat.npy', allow_pickle=True)
ops =  np.load('ops.npy', allow_pickle=True)
ops = ops.item()
iscell = np.load('iscell.npy', allow_pickle=True)

In [ ]:
#select cells and define intermediate values
neurons_raw = F[iscell[:, 0].astype(bool), :]
nN, nFrames = neurons_raw.shape
frameTime = np.arange(1, nFrames + 1) / ops['fs']

In [ ]:
#correct for neuropil and zscore
neuropil = Fneu[iscell[:,0].astype(bool),:]
neurons = F - Fneu
neurons = zscore(neurons, axis=1)

In [ ]:
#smooth
neurons_smooth = gaussian_filter1d(neurons, sigma=1.5, axis=1)
plt.figure(figsize=(12, 6))
plt.plot(frameTime, neurons[10,:], label='Noisy')
plt.plot(frameTime, neurons_smooth[10,:], label='Smoothed', linewidth=2)
plt.legend()
plt.show()

In [ ]:
#remove frame artifacts

# Calculate difference of mean values across columns
dPop = np.diff(np.mean(neuron.F, axis=0), prepend=0)

# Identify artifacts where change is > 50
artefacts = np.abs(dPop) > 50

# Set artifact columns to NaN
neuron.F[:, artefacts] = np.nan
neuron.neuropil[:, artefacts] = np.nan

#define a function to interpolate across the nans
def interpNaN(array):
    array = array.copy()
    nans = np.isnan(array)
    x = nans.nonzero()[0]
    xp = (~nans).nonzero()[0]
    fp = array[~nans]
    array[nans] = np.interp(x, xp, fp)
    return array

# If artifacts found, interpolate NaNs along each row
if np.sum(artefacts) > 0:
    for iN in range(nN):
        neuron.F[iN, :] = interpNaN(neuron.F[iN, :])
        neuron.neuropil[iN, :] = interpNaN(neuron.neuropil[iN, :])


In [ ]:
#debleach?

In [ ]:
#plot first n neurons' z-scored fluorescence traces
n = 5 #how many neurons' traces do you want to visualize
offset = 2 #so the traces don't overlap
plt.figure(figsize=(12, 6))
for i in range(min(n, neurons.shape[0])):
    plt.plot(frameTime, neurons[i] + i * offset, label=f'Neuron {i+1}')
plt.xlabel('Seconds')
plt.ylabel('Z-scored fluorescence')
plt.title('Z-scored fluorescence traces of neurons')
plt.legend()
plt.show()

In [ ]:
#heatmap of many neurons' activity
fluo_interval = 100 #to visualize every nth neuron in the recording
plt.figure(figsize=(10, 8))
plt.imshow(neurons[0::fluo_interval,:], aspect='auto', cmap='viridis')
plt.colorbar(label='Z-scored fluorescence')
plt.xlabel('Frame')
plt.ylabel('Neuron')
plt.title('Heatmap of z-scored neural activity')
plt.show()

In [ ]:
#raster plot of deconvolved spikes
spike_interval = 100 #to visualize every nth neuron in the recording
plt.figure(figsize=(12, 6))
plt.imshow(spks[0::spike_interval,:], aspect='auto', cmap='Greys', interpolation='none')
plt.xlabel('Frame')
plt.ylabel('Neurons')
plt.title('Raster plot')
plt.show()

#But this is hard to make sense of. Let's sort the neurons based on the similarity of their activity.
!pip install rastermap
from rastermap import Rastermap, utils
n_neurons, n_time = spks.shape
print(f"{n_neurons} neurons by {n_time} timepoints")
spks = zscore(spks, axis=1)
model = Rastermap(n_clusters=100, # number of clusters to compute
                  n_PCs=128, # number of PCs to use
                  locality=0.75, # locality in sorting to find sequences (this is a value from 0-1)
                  time_lag_window=5, # use future timepoints to compute correlation
                  grid_upsample=10, # default value, 10 is good for large recordings
                ).fit(spks)
y = model.embedding
isort = model.isort #this is the array we use to sort our data
sorted_data = spks[isort, :]
plt.figure(figsize=(12, 6))
plt.imshow(sorted_data, aspect='auto', cmap='Greys', interpolation='none')
plt.xlabel('Frame')
plt.ylabel('Neurons (sorted)')
plt.title('Raster plot sorted by activity')
plt.show()


In [ ]:
#show ROIs overlaid on the mean image - TEST
mean_image = ops['meanImg']
plt.imshow(mean_image, cmap='gray')

for i, roi in enumerate(stat):
    if iscell[i,0]:
        poly = Polygon([roi['med'[0]], [roi['med'][1]]], fill=None, edgecolor='r', linewidth=1)
        plt.gca().add_patch(poly)
plt.title('ROIs overlay on mean image')
plt.show()